In [1]:
import os
import os.path
import re
import glob
from shutil import copyfile, copy
import pydicom
import xlsxwriter

In [2]:
# 序列重命名每个case
def rename_dir(batch_path):
    # 切换到给定目录下
    os.chdir(batch_path)
    # 序列第一个文件名
    a = 1
    for dir_or_file in os.listdir(batch_path):
        # 筛选得到所有非隐藏的文件夹
        if os.path.isdir(dir_or_file) and not dir_or_file.startswith('.') and not dir_or_file.startswith('pictures'):
            # 重命名文件夹
            os.rename(dir_or_file, str(a))
            a += 1

In [3]:
# 测试rename_dir()
rename_dir('C:\\Users\\Steven\\Desktop\\CT_data\\CT_data3')

FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: '10' -> '2'

In [24]:
# 复制每个case中的jpg文件到pictures文件夹下
def get_pictures(batch_path):
    # 切换到给定目录下
    os.chdir(batch_path)
    # 在当面目录下创建pictures文件夹
    if os.path.exists(r'pictures'):
        pass
    else:
        os.mkdir(r'pictures')
    # 序列图片第一个文件名（需要与rename_dir中文件夹的第一个文件名对应）
    a = 1
    for dir_or_file in os.listdir(batch_path):
        # 得到所有目标文件夹(含有jpg图片的文件夹)
        if os.path.isdir(dir_or_file) and not dir_or_file.startswith(".") and not dir_or_file.startswith('pictures'):
            # 得到当前目标文件夹下所有jpg文件
            for file in glob.glob(dir_or_file + r'/*.jpg'):
                # 新的图片名字
                new_name = dir_or_file +  '/' +  dir_or_file + '.jpg'
                # 重命名图片
                os.rename(file, new_name)
                # 复制jpg文件到pictures文件夹下
                copy(new_name, r'pictures')
                a += 1

In [25]:
# 测试get_pictures()
get_pictures('C:\\Users\\Steven\\Desktop\\CT_data\\CT_data3')

In [4]:
# 给定一个case，递归调用找到任意一个dicom文件及该case的切片数量
def find_dicom(case_path):
    for dir_or_file in os.listdir(case_path):
        dir_or_file_path = case_path + '/' + dir_or_file
        if os.path.isdir(dir_or_file_path):
            return find_dicom(dir_or_file_path)
        else:
            filename = dir_or_file_path
            slice_number = len(os.listdir(case_path))
            return filename, slice_number
        
# 读取dicom文件的患者信息
def get_dicom_info(filename, slice_number, dir_or_file):
    # 建立患者信息字典
    information = {}
    # 读取信息
    ds = pydicom.read_file(filename)
    # 患者id
    information['ID'] = int(dir_or_file)
    # 患者性别
    if ds.PatientSex == 'M':
        information['Sex'] = 'man'
    else:
        information['Sex'] = 'woman'
    try:
        # 患者年龄
        information['Age'] = ds.PatientAge
    except Exception:
        information['Age'] = 0
    # slice_number
    information['Slice_num'] = slice_number
    return information

# 创建一个批次患者的information_list
def get_information_list(batch_path):
    # 创建information_list
    information_list = []
    for dir_or_file in os.listdir(batch_path):
        # 得到case_path
        if os.path.isdir(dir_or_file) and not dir_or_file.startswith(".") and not dir_or_file.startswith('pictures'):
            # 进入每个case_path得到对应的filename和slice_number
            filename, slice_number = find_dicom(os.getcwd() + '/' + dir_or_file + '/' + 'DICOM')
            # 通过得到的filename和slice_number得到患者的information字典
            information = get_dicom_info(filename, slice_number, dir_or_file)
            # 将字典添加到information_list中
            information_list.append(information)
    return information_list

# 将病患信息和切片数量写入excel文件中
def generate_excel(batch_path, excel_path):
    # 创建一个工作簿并添加一个工作表
    wb = xlsxwriter.Workbook(excel_path)
    ws = wb.add_worksheet()
    # 写入表格每一栏标题信息
    ws.write(0, 0, 'No.')
    ws.write(0, 1, 'Sex')
    ws.write(0, 2, 'Age')
    ws.write(0, 3, 'Slice_num')
    ws.write(0, 4, '诊断信息')
    # 得到batch_path下的information_list
    information_list = get_information_list(batch_path)
    # 患者信息的初始行数与列数及编号
    col = 0
    # 得到每个患者的信息
    for information in information_list:
        row = information['ID']
        ws.write(row, col,   information['ID'])
        ws.write(row, col+1, information['Sex'])
        ws.write(row, col+2, information['Age'])
        ws.write(row, col+3, information['Slice_num'])
    # 关闭并保存工作簿
    wb.close()

In [5]:
# 测试generate_excel()
generate_excel('C:\\Users\\Steven\\Desktop\\CT_data\\CT_data3', 'C:\\Users\\Steven\\Desktop\\CT_data\\CT_data3\\third_batch.xlsx')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Steven\\Desktop\\CT_data\\CT_data3\\third_batch.xlsx'